In [1]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_palette('muted')

## Load Data

In [ ]:
eth_raw = pd.read_csv('eth_raw.csv')
print(eth_raw.info())
eth_raw.head()

In [ ]:
eth = eth_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
eth['timestamp'] = pd.to_datetime(eth['timestamp'])
eth['timestamp'] = eth['timestamp'].dt.date
eth.rename(columns = {'timestamp':'date'}, inplace=True)
eth['date'] = pd.to_datetime(eth['date'], format='%Y-%m-%d')
eth.head(2)

In [ ]:
eth.info()

In [ ]:
# save df
eth.to_pickle('eth_df.pkl')

## Exogenous Variables
- Etherem related
    - high
    - low
    - open
    - volume
    - market cap
- Other Cryptocurrencies
    - BTC: Bitcoin
    - XRP: Ripple
    - EOS: EOS.IO
    - LTC: Litecoin
    - XLM: Stellar
    - XMR: Monero
- Other Indices
    - VIX
    - DXY
    - EFFR

In [ ]:
#eth_market_cap = pd.read_csv('ethereum_price.csv')
#print(eth_market_cap.info())
#eth_market_cap.head(2)

In [ ]:
#eth_market_cap['Date'] = pd.to_datetime(eth_market_cap['Date'],
#                                        format='%b %d, %Y')
#eth_market_cap.sort_values('Date', inplace=True)
#eth_market_cap.reset_index(drop=True, inplace=True)
#eth_market_cap['Market Cap'] = pd.to_numeric(
#    eth_market_cap['Market Cap'].str.replace(',', ''),
#    errors='coerce')
#eth_market_cap.rename(columns={'Date':'date', 'Market Cap':'market_cap'}, inplace=True)
#eth_market_cap.head(2)

In [ ]:
#eth_market_cap.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

In [ ]:
#df = eth.merge(eth_market_cap, how='outer', on='date')

In [ ]:
#df.head(2)

### **[VIX: CBOE Volatility Index](https://en.wikipedia.org/wiki/VIX)**
- Measure of stock market's expectation of volatility implied by S&P 500 index options, aka 'fear index'
- [Data Source](http://www.cboe.com/products/vix-index-volatility/vix-options-and-futures/vix-index/vix-historical-data)

In [ ]:
df = eth.copy()

In [ ]:
vix_raw = pd.read_csv('vixcurrent.csv', header=1)
print(vix_raw.info())
vix_raw[:1]

In [ ]:
start_date = '2015-08-06'
end_date = '2018-09-09'
vix_close = vix_raw[['Date', 'VIX Close']].copy()
vix_close['Date'] = pd.to_datetime(vix_close['Date'], format='%m/%d/%Y')
vix_close = vix_close.loc[(vix_close['Date'] >= start_date)
                          & (vix_close['Date'] <= end_date)]
vix_close.sort_values('Date', inplace=True)
vix_close.reset_index(drop=True, inplace=True)
vix_close.columns = ['date', 'vix']
vix_close.head()

In [ ]:
df = df.merge(vix_close, how='outer', on='date')
df.vix.ffill(inplace=True)
#df_04ex.vix[0]= vix_raw.loc[vix_raw['Date'] == '11/24/2017', ['VIX Close']].values
print(df.vix.isnull().sum())
df.head()

### **[DXY: US Dollar Index](https://en.wikipedia.org/wiki/U.S._Dollar_Index)**
- Measure of value of USD relative to a basket of foreign currencies
- [Data Source](https://quotes.wsj.com/index/DXY/historical-prices)

In [ ]:
dxy_raw = pd.read_csv('dxy_historical.csv')
print(dxy_raw.info())
dxy_raw[:1]

In [ ]:
dxy = dxy_raw[['Date', ' Close']].copy()
dxy['Date'] = pd.to_datetime(dxy['Date'], format='%x')
dxy = dxy.loc[(dxy['Date'] >= start_date)
              & (dxy['Date'] <= end_date)]
dxy.sort_values('Date', inplace=True)
dxy.reset_index(drop=True, inplace=True)
dxy.columns = ['date', 'dxy']
dxy.head()

In [ ]:
df = df.merge(dxy, how='outer', on='date')
df.ffill(inplace=True)

# get first value
#dxy_raw['Date'] = pd.to_datetime(dxy_raw['Date'], format='%x')
#df_04ex.dxy[0] = dxy_raw.loc[dxy_raw['Date'] == '11/24/2017', [' Close']].values
print(df.dxy.isnull().sum())
df.head(2)

In [ ]:
df.tail(2)

### **[EFFR: Federal Funds Rate](https://en.wikipedia.org/wiki/Federal_funds_rate)**
- The interest rate at which depository institutions (banks and credit unions) lend reserve balances to other depository institutions overnight, on an uncollateralized basis 
- [Data Source](https://fred.stlouisfed.org/series/EFFR)

In [ ]:
effr_raw = pd.read_csv('EFFR.csv')
effr_raw.head()

In [ ]:
effr = effr_raw.copy()
effr.columns = ['date', 'effr']
effr['date'] = pd.to_datetime(effr['date'], format='%Y-%m-%d')
effr.sort_values('date', inplace=True)
effr = effr.loc[(effr['date'] >= start_date) & (effr['date'] <= end_date)]
effr.reset_index(drop=True, inplace=True)
effr.head()

In [ ]:
df = df.merge(effr, how='outer', on='date')
df.ffill(inplace=True)
print(df.effr.isnull().sum())
df.head()

## Add other coins

In [ ]:
# bitcoin
btc_raw = pd.read_csv('btc_raw.csv')
btc = btc_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
btc['timestamp'] = pd.to_datetime(btc['timestamp'])
btc['timestamp'] = btc['timestamp'].dt.date
btc.rename(columns = {'timestamp':'date', 'close':'btc'}, inplace=True)
btc['date'] = pd.to_datetime(btc['date'], format='%Y-%m-%d')
btc = btc.loc[(btc['date'] >= start_date) & (btc['date'] <= end_date)]
btc.sort_values(by='date', inplace=True)
btc.reset_index(drop=True, inplace=True)
btc.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
btc.head(2)

In [ ]:
df = df.merge(btc, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# ripple
xrp_raw = pd.read_csv('xrp_raw.csv')
xrp = xrp_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xrp['timestamp'] = pd.to_datetime(xrp['timestamp'])
xrp['timestamp'] = xrp['timestamp'].dt.date
xrp.rename(columns = {'timestamp':'date', 'close':'xrp'}, inplace=True)
xrp['date'] = pd.to_datetime(xrp['date'], format='%Y-%m-%d')
xrp = xrp.loc[(xrp['date'] >= start_date) & (xrp['date'] <= end_date)]
xrp.sort_values(by='date', inplace=True)
xrp.reset_index(drop=True, inplace=True)
xrp.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xrp, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# eos.io
eos_raw = pd.read_csv('eos_raw.csv')
eos = eos_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
eos['timestamp'] = pd.to_datetime(eos['timestamp'])
eos['timestamp'] = eos['timestamp'].dt.date
eos.rename(columns = {'timestamp':'date', 'close':'eos'}, inplace=True)
eos['date'] = pd.to_datetime(eos['date'], format='%Y-%m-%d')
eos = eos.loc[(eos['date'] >= start_date) & (eos['date'] <= end_date)]
eos.sort_values(by='date', inplace=True)
eos.reset_index(drop=True, inplace=True)
eos.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(eos, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# litecoin
ltc_raw = pd.read_csv('ltc_raw.csv')
ltc = ltc_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
ltc['timestamp'] = pd.to_datetime(ltc['timestamp'])
ltc['timestamp'] = ltc['timestamp'].dt.date
ltc.rename(columns = {'timestamp':'date', 'close':'ltc'}, inplace=True)
ltc['date'] = pd.to_datetime(ltc['date'], format='%Y-%m-%d')
ltc = ltc.loc[(ltc['date'] >= start_date) & (ltc['date'] <= end_date)]
ltc.sort_values(by='date', inplace=True)
ltc.reset_index(drop=True, inplace=True)
ltc.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(ltc, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# stellar
xlm_raw = pd.read_csv('xlm_raw.csv')
xlm = xlm_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xlm['timestamp'] = pd.to_datetime(xlm['timestamp'])
xlm['timestamp'] = xlm['timestamp'].dt.date
xlm.rename(columns = {'timestamp':'date', 'close':'xlm'}, inplace=True)
xlm['date'] = pd.to_datetime(xlm['date'], format='%Y-%m-%d')
xlm = xlm.loc[(xlm['date'] >= start_date) & (xlm['date'] <= end_date)]
xlm.sort_values(by='date', inplace=True)
xlm.reset_index(drop=True, inplace=True)
xlm.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xlm, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# monero
xmr_raw = pd.read_csv('xmr_raw.csv')
xmr = xmr_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xmr['timestamp'] = pd.to_datetime(xmr['timestamp'])
xmr['timestamp'] = xmr['timestamp'].dt.date
xmr.rename(columns = {'timestamp':'date', 'close':'xmr'}, inplace=True)
xmr['date'] = pd.to_datetime(xmr['date'], format='%Y-%m-%d')
xmr = xmr.loc[(xmr['date'] >= start_date) & (xmr['date'] <= end_date)]
xmr.sort_values(by='date', inplace=True)
xmr.reset_index(drop=True, inplace=True)
xmr.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xmr, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
df.to_pickle('crypto_df.pkl')

In [ ]:
df.tail()

In [ ]:
print(df.xlm.isnull().sum())